***notebook này tập trung sử dụng GAN để tạo sinh thêm dữ liệu mới, fine-tuning lại GAN để có thể tạo sinh dữ liệu mới và phân loại***

***references:***
- https://www.analyticsvidhya.com/blog/2021/04/generate-your-own-dataset-using-gan/
- https://www.impetus.com/resources/blog/synthetic-data-generation-using-gans/
- https://medium.com/the-research-nest/exploring-gans-to-generate-synthetic-data-ca48f8a4b518
- https://towardsdatascience.com/generative-ai-synthetic-data-generation-with-gans-using-pytorch-2e4dde8a17dd

***Nội dung báo cáo:***

0. báo cáo nội dung thực hiện và kết quả
1. sử dụng GAN tạo dữ liệu mới
2. so sánh dữ liệu mới được tạo ra với bộ dữ liệu cũ
3. training lại mô hình với dữ liệu mới và so sánh kết quả

# 0. báo cáo nội dung thực hiện và kết quả

# 1. sử dụng GAN tạo dữ liệu mới

The goals of this tutorial are simple
- Train a GAN on a dataset
- Use the trained generator to create synthetic data
- Train a machine learning model on the synthetic data
- Use the synthetic model on the real data and check how it performs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Concatenate
from keras.models import Model
from keras.optimizers import Adam

In [2]:
csv_path = "E:/IAD/INTERN/01_diabetes/data/diabetes.csv"
df = pd.read_csv(csv_path)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
x = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# normalize the data
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# one-hot encode the labels
oneHotEncoder = OneHotEncoder(sparse_output=False)
oneHotLabels = oneHotEncoder.fit_transform(np.array(y).reshape(-1, 1))

In [4]:
print(x_scaled.shape, oneHotLabels.shape)
print(x_scaled[0])
print(oneHotLabels[0])

(768, 8) (768, 2)
[ 0.63994726  0.84832379  0.14964075  0.90726993 -0.69289057  0.20401277
  0.46849198  1.4259954 ]
[0. 1.]


In [5]:
NUM_FEATURES = 8
NUM_CLASSES = 2
NOISE_DIM = 16 # CON SỐ NÀY LÀ TỰ QUY ĐỊNH HOẶC ĐƯỢC TÌM RA TRONG QUÁ TRÌNH TỐI ƯU MẠNG GAN
BATCH_SIZE = 64
TRANING_STEPS = 5000

In [6]:
def create_generator():
    noise_input = Input(shape=(NOISE_DIM,))
    class_input = Input(shape=(NUM_CLASSES,))
    merged_input = Concatenate()([noise_input, class_input])
    hidden = Dense(128, activation='relu')(merged_input)
    output = Dense(NUM_FEATURES, activation='linear')(hidden) # hàm kích hoạt là linear để phù hợp với dữ liệu thực tế
    model = Model(inputs=[noise_input, class_input], outputs=output)
    return model

In [7]:
def create_discriminator():
    data_input = Input(shape=(NUM_FEATURES,))
    class_input = Input(shape=(NUM_CLASSES,))
    merged_input = Concatenate()([data_input, class_input])
    hidden = Dense(128, activation='relu')(merged_input)
    output = Dense(1, activation='sigmoid')(hidden) # hàm kích hoạt là sigmoid vì đầu ra phân loại nhị phân dựa trên xác suất (nhãn 0 hoặc 1)
    model = Model(inputs=[data_input, class_input], outputs=output)
    return model

In [8]:
def create_cgan(generator, discriminator):
    noise_input = Input(shape=(NOISE_DIM,))
    class_input = Input(shape=(NUM_CLASSES,))
    generated_data = generator([noise_input, class_input])
    validity = discriminator([generated_data, class_input])
    model = Model(inputs=[noise_input, class_input], outputs=validity)
    return model

In [9]:
generator = create_generator()

discriminator = create_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam())
discriminator.trainable = False

gan = create_cgan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

for step in range(TRANING_STEPS):
    # sinh dữ liệu ngẫu nhiên
    noise = np.random.normal(0, 1, size=(BATCH_SIZE, NOISE_DIM))
    # vì là generated_data là fake data nên nhãn là 0
    generated_data_labels = np.zeros((BATCH_SIZE, 2))
    # sinh dữ liệu giả
    generated_data = generator.predict([noise, generated_data_labels])
    

    # lấy mẫu từ dữ liệu thật
    idxs = np.random.randint(0, x_scaled.shape[0], BATCH_SIZE)
    real_data = x_scaled[idxs]
    # do dữ liệu này được láy ra từ dữ liệu thật, nhãn của nó sẽ được quy định là 1 khi được đưa vào huấn luyện discriminator
    # quy ước, nhãn 0 là dữ liệu giả, nhãn 1 là dữ liệu thật
    real_data_labels = np.ones((BATCH_SIZE, 2))
    # mục đích tăng sự ổn định trong quá trình huấn luyên, nhãn thật được nhân với 0.9
    
    # xây dựng batch dữ liệu cho discriminator (gộp dữ liệu giả + thật vào một batch)
    x_batch = np.concatenate([real_data, generated_data], axis=0)
    y_batch = np.concatenate([real_data_labels, generated_data_labels], axis=0)
    y_discriminator = np.zeros(BATCH_SIZE * 2)
    y_discriminator[:BATCH_SIZE] = 0.9
    # ngoài cách gộp batch dữ liệu như trên, cũng có thể huấn luyện lần lượt trên cả hai bộ dữ liệu fake và real

    # huấn luyện discriminator
    discriminator.trainable = True
    discriminator_loss = discriminator.train_on_batch([x_batch, y_batch], y_discriminator)

    # huấn luyện generator thông qua gan
    noise = np.random.normal(0, 1, size=(BATCH_SIZE, NOISE_DIM))
    y_noise = np.ones((BATCH_SIZE, 2))
    discriminator.trainable = False
    generator_loss = gan.train_on_batch([noise, y_noise], np.ones((BATCH_SIZE, 1)))

    # in kết quả sau mỗi 1000 epochs
    if step % 1000 == 0:
        print(f'epoch = {step}, discriminator = {discriminator_loss}, generator = {generator_loss}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
epoch = 0, discriminator = 0.7203350067138672, generator = [array(0.720335, dtype=float32), array(0.720335, dtype=float32)]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step   
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 916us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 998us/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step
2/

In [10]:
def generate_data(generator, data_class, num_instances):
    oneHotClass = oneHotEncoder.transform(np.array([[data_class]]))
    noise_data = np.random.normal(0, 1, size=(num_instances, NOISE_DIM))
    generated_data = generator.predict([noise_data, np.repeat(oneHotClass, num_instances, axis=0)])
    return pd.DataFrame(generated_data, columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'])

In [12]:
# tạo 64 dữ liệu ngẫu nhiên thuộc class 1
# bằng một lí do nào đó, số lượng mẫu nên là bội của 64 -.-'
generated_data = generate_data(generator, 1, 1000)
# print(generated_data)

# transform lại data chuẩn
generated_data = scaler.inverse_transform(generated_data)
df_generated_data = pd.DataFrame(generated_data, columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'])
df_generated_data.head()

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,4.604581,143.396973,38.430855,15.263789,57.064838,24.629074,0.576654,39.366684
1,4.735234,142.227463,38.804237,15.474180,55.961258,24.283031,0.572284,38.515656
2,4.256870,139.352173,37.756615,13.599657,52.239479,24.746943,0.563981,39.096706
3,4.598587,140.562698,37.699867,14.767749,53.433651,24.651016,0.540906,38.518291
4,4.916012,142.241760,37.524017,13.969641,58.468933,24.535431,0.561694,38.606255


In [15]:
df_generated_data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,4.612000,141.484528,37.745575,14.542930,55.742058,24.462988,0.559399,38.757145
std,0.147091,1.180197,0.743397,0.608656,3.192032,0.268360,0.015779,0.366902
min,4.175292,137.414200,35.244793,12.757532,42.862236,23.518702,0.518412,37.696735
25%,4.516222,140.673561,37.232239,14.110502,53.656325,24.284390,0.548533,38.512140
50%,4.615138,141.495934,37.771402,14.548351,55.678440,24.465782,0.557791,38.762104
75%,4.710857,142.294590,38.255103,14.988830,57.896063,24.640278,0.570453,39.013052
max,5.085800,145.516357,39.942699,16.394043,66.994644,25.296854,0.612452,39.943874


In [16]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


# một phiên bản code GAN tương tự nhưng dễ hiểu hơn

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Concatenate
from keras.optimizers import Adam

# Các biến số
NOISE_DIM = 100
NUM_FEATURES = 8
NUM_CLASSES = 2
BATCH_SIZE = 128

# Tạo mô hình Generator
def create_generator():
    noise_input = Input(shape=(NOISE_DIM,))
    class_input = Input(shape=(NUM_CLASSES,))
    merged_input = Concatenate()([noise_input, class_input])
    hidden = Dense(256, activation='relu')(merged_input)
    output = Dense(NUM_FEATURES, activation='sigmoid')(hidden)
    model = Model(inputs=[noise_input, class_input], outputs=output)
    return model

# Tạo mô hình Discriminator
def create_discriminator():
    data_input = Input(shape=(NUM_FEATURES,))
    class_input = Input(shape=(NUM_CLASSES,))
    merged_input = Concatenate()([data_input, class_input])
    hidden = Dense(128, activation='relu')(merged_input)
    output = Dense(1, activation='sigmoid')(hidden)
    model = Model(inputs=[data_input, class_input], outputs=output)
    model.compile(loss='binary_crossentropy', optimizer=Adam())
    return model

# Tạo mô hình cGAN
def create_cgan(generator, discriminator):
    noise_input = Input(shape=(NOISE_DIM,))
    class_input = Input(shape=(NUM_CLASSES,))
    generated_data = generator([noise_input, class_input])
    validity = discriminator([generated_data, class_input])
    model = Model(inputs=[noise_input, class_input], outputs=validity)
    model.compile(loss='binary_crossentropy', optimizer=Adam())
    return model

# Khởi tạo các mô hình
generator = create_generator()
discriminator = create_discriminator()
cgan = create_cgan(generator, discriminator)

# Giả lập dữ liệu cho quá trình huấn luyện
x_real = np.random.rand(BATCH_SIZE, NUM_FEATURES)
y_real = np.random.randint(0, 2, size=(BATCH_SIZE, NUM_CLASSES))

noise = np.random.rand(BATCH_SIZE, NOISE_DIM)
y_fake = np.random.randint(0, 2, size=(BATCH_SIZE, NUM_CLASSES))

# Nhãn cho dữ liệu thật và giả
real_labels = np.ones((BATCH_SIZE, 1))
fake_labels = np.zeros((BATCH_SIZE, 1))

# Huấn luyện Discriminator với dữ liệu thật
d_loss_real = discriminator.train_on_batch([x_real, y_real], real_labels)

# Tạo dữ liệu giả bằng Generator
x_fake = generator.predict([noise, y_fake])

# Huấn luyện Discriminator với dữ liệu giả
d_loss_fake = discriminator.train_on_batch([x_fake, y_fake], fake_labels)

# Tổng mất mát của Discriminator
d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

# Huấn luyện Generator thông qua cGAN
g_loss = cgan.train_on_batch([noise, y_fake], real_labels)

print("Mất mát của Discriminator với dữ liệu thật:", d_loss_real)
print("Mất mát của Discriminator với dữ liệu giả:", d_loss_fake)
print("Mất mát tổng của Discriminator:", d_loss)
print("Mất mát của Generator:", g_loss)


***giải thích:***

Trong Conditional GAN (cGAN), chúng ta có hai đầu vào cho GAN: một là nhiễu (noise) và một là lớp (class) để điều kiện hóa quá trình tạo dữ liệu. Điều này có nghĩa là cả Generator và Discriminator đều nhận thêm thông tin lớp cùng với các đầu vào khác của chúng.

Trong ví dụ này:

    - Generator nhận đầu vào là nhiễu (noise_input) và lớp (class_input). Nó tạo ra dữ liệu giả có cùng số chiều với dữ liệu thật.

    - Discriminator nhận đầu vào là dữ liệu (thật hoặc giả) và lớp (class_input). Nó cố gắng phân biệt giữa dữ liệu thật và giả.

    - cGAN kết hợp Generator và Discriminator. Nó nhận đầu vào là nhiễu và lớp, sau đó sử dụng Generator để tạo dữ liệu giả và Discriminator để đánh giá dữ liệu đó.

Trong quá trình huấn luyện:

    - Discriminator được huấn luyện trước bằng cách sử dụng cả dữ liệu thật và dữ liệu giả. Chúng ta cung cấp cả dữ liệu và lớp tương ứng của chúng.

    - Generator được huấn luyện thông qua cGAN bằng cách tối ưu hóa khả năng "lừa" Discriminator để đánh giá dữ liệu giả là thật.